In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import glob
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
import random


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
# Define paths to your source folders and destination folders
source_folder_0 = '/home/e17358/4yp/MedNET/dataset/Breast Histopathology Images new/0'
source_folder_1 = '/home/e17358/4yp/MedNET/dataset/Breast Histopathology Images new/1'
base_dest_folder = '/home/e17358/4yp/MedNET/dataset/Breast Histopathology Images new/'

# Define the split ratios
train_ratio = 0.7  # 70% of data for training
valid_ratio = 0.15  # 15% of data for validation
test_ratio = 0.15  # 15% of data for testing

# Create destination folders if they don't exist
os.makedirs(base_dest_folder, exist_ok=True)

# List the images in the source folders
images_0 = os.listdir(source_folder_0)
images_1 = os.listdir(source_folder_1)

# Shuffle the lists of images
random.shuffle(images_0)
random.shuffle(images_1)

# Split the images and move them to the appropriate folders
def move_images(images, dest_folder,source):
    for image in images:
        source_path = os.path.join(source, image)
        if os.path.isdir(dest_folder) is False:
            os.makedirs(dest_folder)
        shutil.move(source_path, dest_folder)


# Split and move images for class 0
train_count = int(len(images_0) * train_ratio)
valid_count = int(len(images_0) * valid_ratio)

move_images(images_0[:train_count], os.path.join(base_dest_folder, 'train', '0'),source_folder_0)
move_images(images_0[train_count:train_count + valid_count], os.path.join(base_dest_folder, 'valid', '0'),source_folder_0)
move_images(images_0[train_count + valid_count:], os.path.join(base_dest_folder, 'test', '0'),source_folder_0)

# Split and move images for class 1
train_count = int(len(images_1) * train_ratio)
valid_count = int(len(images_1) * valid_ratio)
move_images(images_1[:train_count], os.path.join(base_dest_folder, 'train', '1'),source_folder_1)
move_images(images_1[train_count:train_count + valid_count], os.path.join(base_dest_folder, 'valid', '1'),source_folder_1)
move_images(images_1[train_count + valid_count:], os.path.join(base_dest_folder, 'test', '1'),source_folder_1)

print("Data splitting and moving complete.")


In [ ]:
# Load pre-trained VGG16 model without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data preprocessing and augmentation``
datagen = ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_generator = datagen.flow_from_directory(
    '/home/e17358/4yp/MedNET/dataset/Breast Histopathology Images new/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = datagen.flow_from_directory(
    '/home/e17358/4yp/MedNET/dataset/Breast Histopathology Images new/valid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
# Fine-tune the model
model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=valid_generator, validation_steps=len(valid_generator))

In [ ]:
# Save the trained model
model.save('medical_image_transfer_learning.h5')